# Combining Results in ORD

This notebook provides a proof of concept example for combining catastrophe
loss model results in the Open Results Data (ORD) format. We follow the
methodology outlined in *Combining_results_in_ORD_v1.1.pdf*.

This notebook is split into the workflow sequence as follows:

1. Load and Group
2. Period Sampling
3. Loss Sampling
4. Output Preparation

In [ ]:
# make sure relative imports work  
# todo: remove after packaging
import os
import sys
module_path = os.path.abspath(os.path.join('..'))


## 1. Load and Group
### Creating Analysis and OutputSet
In this section we create the objects required prior to grouping, namely:
- Analysis table which contains the meta data from the analyses
- OutputSet table which contains references to the ORD results.

The `Analysis` and `OutputSet` dataclasses are defined in `common.py`.

In [5]:
from ord_combining.common import Analysis, OutputSet

?Analysis
?OutputSet

In [2]:
from ord_combining import common

ModuleNotFoundError: No module named 'ord_combining'

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

In [4]:
module_path

'/home/vinulw/code/ODS_Tools'

In [5]:
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from ord_combining.common import